In [6]:
import requests
import pandas as pd
import gzip
import boto3
from datetime import datetime
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from botocore import UNSIGNED
from botocore.config import Config
from typing import List, Optional, Type
from agent_tools.utils import get_openaq_api_key
import requests
from math import cos, radians
anonymous_session = boto3.Session()  # For public bucket
# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

from crewai import LLM

# LLM Configuration
llm = LLM(
    model="openai/gpt-4o-mini",  # call model by provider/model_name
    temperature=0.7,  # Slightly lower temperature for more focused analysis
    max_tokens=1000,  # Increased max tokens for a more comprehensive report
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)


In [10]:
class BoundingBoxExtractorTool(BaseTool):
    """Tool to extract the bounding box coordinates (south, north, west, east) for a given location name using Nominatim."""

    name: str = "bounding_box_extractor"
    description: str = "Extracts the bounding box coordinates (south, north, west, east) for a given location name."
    parameters: Optional[list[dict]] = [
        {
            "name": "location",
            "type": "string",
            "description": "The name of the location to find the bounding box for.",
            "required": True,
        }
    ]
    return_direct: bool = False

    def _expand_bounding_box(self, south_lat, west_lon, north_lat, east_lon, km_expansion=50):
        """Expand the bounding box by a fixed distance (in kilometers)."""
        # Approximate degrees of latitude and longitude for the given expansion
        lat_offset = km_expansion / 111  # 1 degree latitude ≈ 111 km
        lon_offset = km_expansion / (111 * cos(radians((south_lat + north_lat) / 2)))  # Adjust longitude by latitude

        # Expand bounding box
        return [
            south_lat - lat_offset,  # South
            west_lon - lon_offset,  # West
            north_lat + lat_offset,  # North
            east_lon + lon_offset   # East
        ]
        
    def _run(self, location: str) -> list[str] | str:
        """Executes the tool to retrieve the bounding box."""
        url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json&addressdetails=1"
        headers = {"User-Agent": "CrewAI Tool (vishrajagopalan@gmx.com)"}
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            print(data)
            if data:
                bbox = data[0]['boundingbox']
                print(f"Location: {location} ####### Bounding Box: {bbox}")
                south_lat = float(bbox[0])
                north_lat = float(bbox[1])
                west_lon = float(bbox[2])
                east_lon = float(bbox[3])

                # Expand the bounding box
                expanded_bbox = self._expand_bounding_box(south_lat, west_lon, north_lat, east_lon, km_expansion=15)
                print(f"Expanded Bounding Box: {expanded_bbox}")
                return expanded_bbox            
            else:
                return f"Bounding box not found for location: {location}"
        except requests.exceptions.RequestException as e:
            return f"Error fetching bounding box for {location}: {e}"


In [11]:


class OpenMeteoWeatherInput(BaseModel):
    """Input for the OpenMeteoHistoricalWeatherTool using a bounding box."""
    # Bounding box format: [south_latitude, west_longitude, north_latitude, east_longitude]
    bounding_box: List[float] = Field(..., description="Bounding box coordinates in [south_lat, west_lon, north_lat, east_lon] format.")
    start_date: str = Field(..., description="The start date for historical weather data in YYYY-MM-DD format.")
    end_date: str = Field(..., description="The end date for historical weather data in YYYY-MM-DD format.")

# A bounding box, or bbox, is a rectangular area defined by four coordinates: 
# two representing the southwest corner (minimum longitude, minimum latitude) and 
# two representing the northeast corner (maximum longitude, maximum latitude). 
# It's essentially a way to define a geographical area on a map or within a dataset. 

class HistoricalWeatherTool(BaseTool):
    name: str = "HistoricalWeatherTool"
    description: str = (
        "Retrieves DAILY historical weather data (mean temperature, max temperature, min temperature, "
        "sum of precipitation, mean wind speed, mean relative humidity) for a location "
        "defined by a bounding box and a date range from Open-Meteo.com (no API key required for non-commercial use). "
        "The tool internally calculates the center point of the bounding box for the API query."
    )
    args_schema: Type[BaseModel] = OpenMeteoWeatherInput

    def _run(self, bounding_box: List[float], start_date: str, end_date: str) -> str:
        if len(bounding_box) != 4:
            return "Error: Bounding box must contain exactly 4 float values: [south_lat, west_lon, north_lat, east_lon]."

        # bounding_box is in [south_lat, west_lon, north_lat, east_lon]
        south_lat, west_lon, north_lat, east_lon = bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]
        
        # Calculate the center point of the bounding box
        center_latitude = (south_lat + north_lat) / 2
        center_longitude = (west_lon + east_lon) / 2

        base_url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": center_latitude,
            "longitude": center_longitude,
            "start_date": start_date,
            "end_date": end_date,
            "daily": "temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_mean,relative_humidity_2m_mean",
            "timezone": "auto" # It's good practice to specify timezone for daily data
        }

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            if "daily" not in data:
                return f"No daily weather data found for the bounding box {bounding_box} between {start_date} and {end_date}. API response: {data}"

            daily_data = data["daily"]
            times = daily_data.get("time", [])
            temp_means = daily_data.get("temperature_2m_mean", [])
            temp_maxs = daily_data.get("temperature_2m_max", [])
            temp_mins = daily_data.get("temperature_2m_min", [])
            precipitations = daily_data.get("precipitation_sum", [])
            wind_speeds_mean = daily_data.get("wind_speed_10m_mean", [])
            humidities_mean = daily_data.get("relative_humidity_2m_mean", [])

            weather_summary = []
            for i in range(len(times)):
                summary = {
                    "date": times[i],
                    "temperature_mean_2m": temp_means[i] if i < len(temp_means) else "N/A",
                    "temperature_max_2m": temp_maxs[i] if i < len(temp_maxs) else "N/A",
                    "temperature_min_2m": temp_mins[i] if i < len(temp_mins) else "N/A",
                    "precipitation_sum": precipitations[i] if i < len(precipitations) else "N/A",
                    "wind_speed_10m_mean": wind_speeds_mean[i] if i < len(wind_speeds_mean) else "N/A",
                    "relative_humidity_2m_mean": humidities_mean[i] if i < len(humidities_mean) else "N/A"
                }
                weather_summary.append(summary)

            return str(weather_summary) # Return as string for LLM processing
        except requests.exceptions.RequestException as e:
            return f"Error fetching weather data from Open-Meteo for bounding box {bounding_box}: {e}"
        except Exception as e:
            return f"An unexpected error occurred: {e}"
# def main(location: str, start_date: str, end_date: str):
#     """
#     Main function to get the bounding box for a location and then fetch historical weather details.

#     Args:
#         location (str): The name of the location.
#         start_date (str): The start date for weather data in YYYY-MM-DD format.
#         end_date (str): The end date for weather data in YYYY-MM-DD format.
#     """
#     bbox_extractor = BoundingBoxExtractorTool()
#     weather_tool = HistoricalWeatherTool()

#     print(f"Getting bounding box for {location}...")
#     bounding_box = bbox_extractor.run(location=location)

#     if isinstance(bounding_box, str):
#         print(f"Error: {bounding_box}")
#         return

#     print(f"Bounding box for {location}: {bounding_box}")
#     print(f"Fetching weather details for {location} from {start_date} to {end_date}...")
#     weather_details = weather_tool.run(bounding_box=bounding_box, start_date=start_date, end_date=end_date)
    
#     print("\nWeather Details:")
#     print(weather_details)

# if __name__ == "__main__":
#     # Example usage:
#     location_name = "Chennai"
#     start = "2023-01-01"
#     end = "2023-01-03"
#     main(location_name, start, end)

#     print("\n" + "="*50 + "\n")

#     location_name_2 = "London"
#     start_2 = "2024-05-15"
#     end_2 = "2024-05-17"
#     main(location_name_2, start_2, end_2)

In [12]:

class AirQualityAnalysisTool(BaseTool):
    name: str = "air_quality_analysis"
    description: str = "Fetch air quality data for specified locations and dates, returning aggregated results."
    parameters: Optional[List[dict]] = [
        {
            "name": "bounding_boxes",
            "type": "list[list[float]]",
            "description": (
                "List of bounding boxes (each as [south, north, west, east]) to analyze air quality for."
            ),
            "required": True,
        },      
        {
            "name": "locations",
            "type": "list[str]",
            "description": "List of location names to analyze air quality for.",
            "required": True,
        },          
        {
            "name": "start_date",
            "type": "str",
            "description": "Start date for the analysis in YYYY-MM-DD format.",
            "required": True,
        },
        {
            "name": "end_date",
            "type": "str",
            "description": "End date for the analysis in YYYY-MM-DD format.",
            "required": True,
        },
        {
            "name": "aq_parameters",
            "type": "Optional[list[str]]",
            "description": "Optional list of air quality parameters to filter (e.g., ['pm25', 'o3']). If None, all available parameters are used.",
            "required": False,
        },
    ]
    def _run(self, bounding_boxes: List[List[float]], locations: List[str], start_date: str, end_date: str, aq_parameters: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Args:
            bounding_boxes (list): List of bounding boxes as [south, north, west, east].
            locations (list): List of location names corresponding to bounding boxes.            
            start_date (str): Start date for the analysis in YYYY-MM-DD format.
            end_date (str): End date for the analysis in YYYY-MM-DD format.
            aq_parameters (list, optional): List of air quality parameters to filter. Defaults to None.

        Returns:
            pd.DataFrame: Aggregated air quality data with columns [date, parameter, unit, value, location_name].
        """
        

        # Convert start_date and end_date strings to datetime objects
        try:
            start_date_dt = datetime.strptime(start_date, "%Y-%m-%d").date()
            end_date_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
        except ValueError:
            raise ValueError("Invalid date format. Please use YYYY-MM-DD.")

        # Step 2: Fetch location IDs from OpenAQ
        def get_location_ids(bbox: List[str]) -> List[dict]:
            url = "https://api.openaq.org/v3/locations?limit=100&page=1&order_by=id&sort_order=asc"
            params = {
                "bbox": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
            }
            headers = {"X-API-Key": get_openaq_api_key()}
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            return response.json().get("results", [])

        # Step 3: Fetch data from OpenAQ AWS bucket using boto3
        def fetch_sensor_data(location_ids: List[int], start_date: datetime.date, end_date: datetime.date) -> pd.DataFrame: # Updated type hints
            consolidated_df = pd.DataFrame()
            failed_locations = []  # To track locations that fail to return data

            s3_client = anonymous_session.client('s3', region_name="us-east-1", config=Config(signature_version=UNSIGNED))
            source_bucket_name = "openaq-data-archive"

            for location_id in location_ids:
                for date in pd.date_range(start=start_date, end=end_date):
                    year, month, day = date.strftime("%Y"), date.strftime("%m"), date.strftime("%d")
                    prefix = f"records/csv.gz/locationid={location_id}/year={year}/month={month}/"
                    try:
                        response = s3_client.list_objects_v2(Bucket=source_bucket_name, Prefix=prefix)
                        if 'Contents' in response:
                            for obj in response['Contents']:
                                key = obj['Key']
                                if key.endswith(f"{year}{month}{day}.csv.gz"):
                                    print(f"Downloading: {key}")
                                    obj_data = s3_client.get_object(Bucket=source_bucket_name, Key=key)
                                    with gzip.GzipFile(fileobj=obj_data['Body']) as gz_file:
                                        daily_df = pd.read_csv(gz_file)
                                        consolidated_df = pd.concat([consolidated_df, daily_df], ignore_index=True)
                        else:
                            failed_locations.append(location_id)

                    except Exception as e:
                        print(f"Error fetching data for location ID {location_id}: {e}")
                        failed_locations.append(location_id)
            print("Sample Sensor Data from OPENAQ : \n", consolidated_df.head())
            if failed_locations:
                print(f"Locations with no data or errors: {failed_locations}")

            return consolidated_df

        # Step 4: Aggregate data
        def aggregate_data(df: pd.DataFrame, parameters: Optional[List[str]] = None) -> pd.DataFrame:
            df['datetime'] = pd.to_datetime(df['datetime'])  # Ensure datetime is parsed correctly
            df = df.set_index('datetime')  # Set 'datetime' as the index

            if parameters:
                df = df[df['parameter'].isin(parameters)]  
            try : 
                # Reset the index to avoid conflicts with 'parameter'
                df = df.reset_index()     
                print("DF :\n ", df.head())           
                daily_data = (
                    df.groupby(['parameter', pd.Grouper(key='datetime', freq='D')])
                    .agg(
                        value=('value', 'mean'),
                        units=('units', 'first')
                    )
                    .dropna()
                    .reset_index()
                )  
            except Exception as e : 
                print("Aggregation failed with Exception: ", e)
            daily_data['date'] = daily_data['datetime'].dt.date
            del daily_data['datetime']
            return daily_data

        # Main Workflow
        all_data = []
        for bbox, location in zip(bounding_boxes, locations):
            try:
                bbox_openaq_format  = [bbox[1], bbox[0], bbox[3], bbox[2]]         
                location_data = get_location_ids(bbox_openaq_format)
                location_ids = [loc['id'] for loc in location_data]
                # Open AQ accepts bounding box in the format west, south, east, north

                print(f"Found {len(location_ids)} locations for bounding box (per openAQ format) {bbox_openaq_format} (Location: {location}). Downloading data...")

                consolidated_df = fetch_sensor_data(location_ids, start_date_dt, end_date_dt)
                if not consolidated_df.empty:               
                    aggregated_daily_data = aggregate_data(consolidated_df, aq_parameters)
                    aggregated_daily_data["location"] = location  # Add the location column
                    all_data.append(aggregated_daily_data)

            except Exception as e:
                print(f"Error processing bounding box {bbox} for location {location}: {e}")

        # Combine all data
        if all_data:
            result = pd.concat(all_data, ignore_index=True)
            return result
        else:
            return pd.DataFrame(columns=["date", "parameter", "unit", "value", "location"])
        

tool = BoundingBoxExtractorTool()

if __name__ == '__main__':
    # Example usage:
    analysis_tool = AirQualityAnalysisTool()
    locations_to_analyze = [ "New Delhi, India"]
    parameters_to_analyze=["pm25"]
    bboxes=[]
    for location in locations_to_analyze : 
        bbox = tool.run(location=location)
        print(  "Bounding box : ", bbox)
        bboxes.append(bbox)


    start = "2023-01-01"
    end = "2023-01-03"
    parameters_to_analyze = ["pm25"]
    print(bboxes)
    try:
        
        results_df = analysis_tool.run(bounding_boxes = bboxes,locations=locations_to_analyze, start_date=start, end_date=end, aq_parameters=parameters_to_analyze)
        print("\nAggregated Air Quality Data:")
        print(results_df)
    except Exception as e:
        print(f"An error occurred during analysis: {e}")



Using Tool: bounding_box_extractor
[{'place_id': 226589616, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 80414558, 'lat': '28.6430858', 'lon': '77.2192671', 'class': 'railway', 'type': 'station', 'place_rank': 30, 'importance': 0.45936793322640973, 'addresstype': 'railway', 'name': 'New Delhi', 'display_name': 'New Delhi, Foot Over Bridge 1, Ram Nagar, New Delhi Railway Station, Delhi, Kotwali Tehsil, Central Delhi, Delhi, 110006, India', 'address': {'railway': 'New Delhi', 'road': 'Foot Over Bridge 1', 'neighbourhood': 'Ram Nagar', 'suburb': 'New Delhi Railway Station', 'city': 'Delhi', 'ISO3166-2-lvl15': 'IN-DL', 'county': 'Kotwali Tehsil', 'state_district': 'Central Delhi', 'state': 'Delhi', 'ISO3166-2-lvl4': 'IN-DL', 'postcode': '110006', 'country': 'India', 'country_code': 'in'}, 'boundingbox': ['28.6421417', '28.6438360', '77.2189812', '77.2196548']}, {'place_id': 226589546, 'licence': 'Data © OpenStreetMap contr

In [17]:
#creating the Agent Workflow
import os
from crewai import Crew, Agent, Task
from typing import List, Optional
from crewai_tools import ScrapeWebsiteTool




# Load API Keys (ensure these are set as environment variables or securely managed)
openai_api_key = os.environ.get("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

# Initialize Tools
bounding_box_extractor_tool = BoundingBoxExtractorTool()
air_quality_tool = AirQualityAnalysisTool()
weather_tool = HistoricalWeatherTool()


def create_air_quality_analysis_crew(locations: List[str], start_date: str, end_date: str, aq_parameters: Optional[List[str]] = None):
    """Creates and runs the air quality analysis crew."""

    # Agent 1: Bounding Box Retriever
    bounding_box_retriever = Agent(
        role="Geospatial Data Specialist",
        goal="Retrieve bounding box coordinates for the specified locations.",
        backstory="Expert in geographical information retrieval and spatial data analysis.",
        verbose=True,
        allow_delegation=False,
        tools=[bounding_box_extractor_tool],
    )

    # Task 1: Get Bounding Boxes
    get_bounding_boxes_task = Task(
        description=f"For each of the following locations: {locations}, use the 'bounding_box_extractor' tool to find their bounding box coordinates. Return the bounding boxes associated with each location.",
        agent=bounding_box_retriever,
        expected_output="A dictionary or list containing the bounding box coordinates (south, west, north, east) for each specified location.",
    )

    # Agent 2: Weather Data Integrator
    weather_data_integrator = Agent(
        role="Historical Weather Data Specialist",
        goal="Retrieve concise historical weather summaries for the specified locations and dates.",
        backstory="Expert in accessing and summarizing historical meteorological data relevant to environmental analysis.",
        verbose=True,
        allow_delegation=False,
        tools=[weather_tool],
    )

    # Task 2: Get Weather Data
    get_weather_data_task = Task(
        description=f"For each of the following locations: {locations}, use the bounding boxes (south, west, north, east) to query the weather tool to find a concise summary of relevant historical weather conditions between {start_date} and {end_date}. Focus on key weather aspects that might influence air quality (e.g., temperature, wind, precipitation).",
        agent=weather_data_integrator,
        expected_output="A dictionary or list containing concise summaries of historical weather conditions for each specified city.",
        context=[get_bounding_boxes_task], 
    )

    # Agent 3: Air Quality Data Retriever
    air_quality_retriever = Agent(
        role="Air Quality Data Retriever",
        goal="Fetch air quality data from OpenAQ for the specified locations and date range.",
        backstory="Specialized in accessing and retrieving air quality data from the OpenAQ database.",
        verbose=True,
        allow_delegation=False,
        tools=[air_quality_tool],
    )
    
    # Task 3: Get Air Quality Data
    get_air_quality_data_task = Task(
        description=f"Fetch air quality data using the air_quality_tool for the following locations: {locations} from {start_date} to {end_date} using the bounding boxes for each location. If specific parameters are provided ({aq_parameters}), focus on those. Return the data as a pandas DataFrame.",
        agent=air_quality_retriever,
        expected_output="A pandas DataFrame containing the air quality data for the specified locations, dates, and parameters.",
        context=[get_bounding_boxes_task],  # The AirQualityAnalysisTool needs the locations
    )

    # Agent 4: Air Quality Analyst
    air_quality_analyst = Agent(
        role="Air Quality Analyst",
        goal="Analyze the collected air quality data and the corresponding weather information to generate a comprehensive report on the air quality situation.",
        backstory="Experienced environmental scientist specializing in air pollution analysis and its relationship with meteorological conditions.",
        verbose=True,
        allow_delegation=False,
        llm=llm,  # Use the configured LLM
        context=[get_air_quality_data_task] + [get_weather_data_task],
    )

    # Task 4: Analyze and Report
    analysis_task = Task(
        description="Analyze the provided air quality data (including parameters like pm10, value, units, date, and location) for the specified locations and dates. Consider the historical weather information (temperature, wind, precipitation, humidity) for the same period. Identify any trends in air quality, calculate average values where relevant, and discuss any potential correlations or influences of weather conditions on the air quality. Provide a detailed report summarizing the air quality situation for each city, including the key findings and any notable observations related to weather patterns.",
        agent=air_quality_analyst,
        expected_output="A comprehensive report detailing the air quality analysis for each city, including trends, averages, and a discussion of potential relationships with the historical weather conditions.",
    )


    # Instantiate the Crew
    agents = [bounding_box_retriever, weather_data_integrator, air_quality_retriever, air_quality_analyst]
    tasks = [get_bounding_boxes_task, get_weather_data_task, get_air_quality_data_task, analysis_task]


    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=True,
    )

    # Run the crew and get the analysis report
    report = crew.kickoff()
    return report

if __name__ == "__main__":
    locations = ["New Delhi, India", "Chennai, India"]
    analysis_start_date = "2024-12-31"
    analysis_end_date = "2025-01-02"
    parameters_of_interest = ["pm25, pm10"]  # Optional

    try:
        analysis_report = create_air_quality_analysis_crew(
            locations=locations,
            start_date=analysis_start_date,
            end_date=analysis_end_date,
            aq_parameters=parameters_of_interest
        )
        print("\n--- Air Quality Analysis Report ---")
        print(analysis_report)
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9e61d9f6-8813-41a1-95c2-03db69c47831                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 949981bf-7b60-47df-8841-c79c871903cd
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 949981bf-7b60-47df-8841-c79c871903cd
       Status: Executing Task...
    └── 🤖 Agent: Geospatial Data Specialist
            Status: In Progress

# Agent: Geospatial Data Specialist
## Task: For each of the following locations: ['New Delhi, India', 'Chennai, India'], use the 'bounding_box_extractor' tool to find their bounding box coordinates. Return the bounding boxes associated with each location.


🤖 Agent: Geospatial Data Specialist
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 949981bf-7b60-47df-8841-c79c871903cd
│      Assigned to: Geospatial Data Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Geospatial Data Specialist
│           Status: ✅ Completed
├── 📋 Task: dfa17cc8-5fbd-4761-b884-2960eefd804b
│      Assigned to: Historical Weather Data Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Historical Weather Data Specialist
│           Status: ✅ Completed
├── 📋 Task: 17409821-f67b-4c59-bee5-c21170612783
│      Assigned to: Air Quality Data Retriever
│      Status: ✅ Completed
│   └── 🤖 Agent: Air Quality Data Retriever
│           Status: ✅ Completed
└── 📋 Task: aec9dcf3-eacf-4ceb-a571-3063ee5e9060
       Status: Executing Task...
    └── 🤖 Agent: Air Quality Analyst
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 949981bf-7b60-47df-8841-c79c871903cd
│      Assigned to: Geospatial Data Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Geospatial Data Specialist
│           Status: ✅ Completed
├── 📋 Task: dfa17cc8-5fbd-4761-b884-2960eefd804b
│      Assigned to: Historical Weather Data Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Historical Weather Data Specialist
│           Status: ✅ Completed
├── 📋 Task: 17409821-f67b-4c59-bee5-c21170612783
│      Assigned to: Air Quality Data Retriever
│      Status: ✅ Completed
│   └── 🤖 Agent: Air Quality Data Retriever
│           Status: ✅ Completed
└── 📋 Task: aec9dcf3-eacf-4ceb-a571-3063ee5e9060
       Assigned to: Air Quality Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Air Quality Analyst
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aec9dcf3-eacf-4ceb-a571-3063ee5e9060                                                                     │
│  Agent: Air Quality Analyst                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9e61d9f6-8813-41a1-95c2-03db69c47831                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Air Quality Analysis Report ---
**Comprehensive Air Quality and Weather Analysis Report**

**1. New Delhi, India**

*Air Quality Data Analysis:*
- PM10 Levels:
  - 2024-12-31: 209.60 µg/m³
  - 2025-01-01: 251.26 µg/m³
  - 2025-01-02: 247.84 µg/m³
- PM2.5 Levels:
  - 2024-12-31: 185.03 µg/m³
  - 2025-01-01: 224.33 µg/m³
  - 2025-01-02: 213.30 µg/m³

*Average Values:*
- PM10 Average: (209.60 + 251.26 + 247.84) / 3 = 236.57 µg/m³
- PM2.5 Average: (185.03 + 224.33 + 213.30) / 3 = 207.22 µg/m³

*Weather Conditions Analysis:*
- Historical Weather Data:
  - 2024-12-31: Mean Temp: 11.2°C, Max Temp: 16.0°C, Min Temp: 7.6°C, Precipitation: 0.0 mm, Wind Speed: 3.0 m/s, Humidity: 89%
  - 2025-01-01: Mean Temp: 11.0°C, Max Temp: 17.0°C, Min Temp: 7.6°C, Precipitation: 0.0 mm, Wind Speed: 5.2 m/s, Humidity: 89%
  - 2025-01-02: Mean Temp: 12.3°C, Max Temp: 18.3°C, Min Temp: 8.3°C, Precipitation: 0.0 mm, Wind Speed: 5.7 m/s, Humidity: 86%

*Trends and Correlations:*
The air quality in New Delhi s